In [1]:

import random
from random import choice, randrange
import matplotlib.image as mpimg

import numpy as np
import tensorflow as tf
from datetime import datetime as dt
import io
import os
from timeit import default_timer as timer
import matplotlib.pyplot as plt



In [2]:
import pathlib

In [3]:
import drawing

In [4]:
#!pip install tensorflow-probability==0.11.0

In [5]:
import tensorflow_probability as tfp
tfd = tfp.distributions

In [6]:
import collections

In [7]:
#!pip install --user tensorflow==2.3

In [8]:
print(tf.__version__)

2.2.0


In [9]:
#tf.debugging.enable_check_numerics()
#tf.config.set_visible_devices([], 'GPU') 
print(tf.config.list_physical_devices('GPU'))
# print(tf.test.is_built_with_cuda())

[]


In [10]:

np.random.seed(121)
tf.random.set_seed(11)

In [14]:
TRAIN_DATA_URL = "C:/Users/Youssef/workspacePy/data_source/medicament_data/med_output2.csv"

In [15]:

def read_med_csv(filename):
    l_med=[]
    with open(filename, 'r',encoding="utf-8" ,newline='') as f:
        lis = f.readlines()
        
        for  i ,line in enumerate(lis):
            if i == 0 :
                continue
            
            record = line.strip()
           
            l_med.append(record.capitalize()+' ')     
                
    return l_med    

In [16]:
med_ds = read_med_csv(TRAIN_DATA_URL)
print(len(set(med_ds)))
med_ds = list(set(med_ds))
print(med_ds[1524].strip())
#med_ds[0] = 'The Shawshank Redemption '
# med_ds[1] = 'Doliprane 500 '
# med_ds[2] = 'Augmentin 1g '

2378
Ervevax


In [17]:

list_med_to_num = [[drawing.alpha_to_num[ch] for ch in me]  for me in med_ds]
real_seqs = [tf.convert_to_tensor(li )  for li in list_med_to_num]



In [18]:
#print(real_seqs[1524])

In [19]:
print(drawing.alphabet)
vocab_size = len( drawing.alphabet )
vocab_size

['\x00', ' ', '!', '"', '%', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


76

In [20]:
#load real_seqs from seved npy file (real_seqs_medicament2_saved.npy)to conserve the same order of words

   # real_seqs = [tf.one_hot(tf.cast(line , dtype = "int32") , vocab_size) for line in real_seqs ]
   # print(real_seqs[0].shape)
real_seqs = np.load('real_seqs_medicament2_saved.npy' , allow_pickle=True)


(11, 76)


In [ ]:
real_seqs = np.load('real_seqs_medicament2_saved.npy' , allow_pickle=True)


In [21]:
path_weight = 'true_data+layerGMM_Q_Z_16_writer_cholski_timedistributed_oneCell_clipGradient' # for trained weights go to folder : true_data+layerGMM'

path_tosave_image = 'my_handwriting_true_data_writer_oneCell_clipgradient/'

vocab_size = len( drawing.alphabet )

units = 400
k_mixture = 10
GMM_mixtures = 20

seq_length = 1

BATCH_SIZE = 1

lst_alpha = []
lst_logprob = []


epochs =50
take_batch =725
custom_period = 20
first_log = True
periodic_log = False
last_batch = False
number_of_writer = 198

In [22]:

len(real_seqs)

2378

In [23]:

class lstm_attention_layer(tf.keras.layers.Layer):
 def __init__(self ,num_cells , kmixtures ):
      super(lstm_attention_layer , self).__init__()
#       self.dense_attention = tf.keras.layers.Dense(kmixtures * 3 , activation=tf.nn.softplus,
#                                                    kernel_initializer = tf.initializers.GlorotUniform(31))#,input_shape =(BATCH_SIZE,seq_length) )
      
      self.lstm = tf.keras.layers.LSTMCell(num_cells, name='lstm_1')
        
        
     
        
      self.dense_attention_a= tf.keras.layers.Dense(kmixtures * 3 , activation=tf.keras.activations.softplus,
                                )

    
      self.mixtures = kmixtures
     
      self.state_size = [num_cells, num_cells , vocab_size , kmixtures]
        
      
# @tf.function       
 def call (self , inpu, states ,constants ):
      global last_batch
      global periodic_log
      
    
    
      prev_h_lstm, prev_c_lstm, prev_w , prev_kappa = states

      lstm_out, lstm_states = self.lstm(tf.concat([inpu, prev_w], axis=-1), (prev_h_lstm, prev_c_lstm))
      h_lstm, c_lstm = lstm_states
        
     
      character_seq = constants[0]#tf.concat( [tf.zeros( (BATCH_SIZE ,char_real_seqs , 1)), whitout_zero] , axis = -1)
      #print('character_seq reduce++++++++++++++++++++++++++++++++' ,character_seq )#tf.reduce_sum(character_seq , axis = -1))
      #print('where_zero' , whitout_zero)
      
      #out_dense = self.dense_attention(tf.concat( [inputs ,states[0] ] ,-1))
      #alpha, beta, kappa   = tf.split(out_dense, 3 ,1)
      
      inputs = tf.concat( [inpu,lstm_out ,constants[1] ] ,-1)
      window_out = self.dense_attention_a( inputs )#(BS , kmixtures * 3  )
      alpha , beta , kappa =  tf.split(window_out, 3, -1)
#       alpha = self.dense_attention_a( inputs )#(BS , kmixtures  )
#       beta = self.dense_attention_b(inputs)#(BS , kmixtures  )

#       kappa = self.dense_attention_k(inputs )#(BS , kmixtures  ) 
     
      if  periodic_log :
            print('inputs to attention cell after concat' , inputs )
            print("constants to attention cell :constants[0] shape" ,constants[0].shape)
            print('states [2]' ,states[2])
            print('states [3]' ,states[3])
            print('kappa ' , kappa) 
            print('beta '   , beta )
            print('alpha '  , alpha)
      kappa_t = prev_kappa + kappa   # states[3] - kappa 
       
      beta_t  =  tf.clip_by_value( beta, .01, 1000 )#tfp.math.clip_by_value_preserve_gradient( beta, .01, 1000 )
      alpha_t =  tf.clip_by_value(alpha, .01, 1000)
      
      window , phi = self.get_window(alpha_t, beta_t, kappa_t ,character_seq) 
      new_phi.assign(tf.cast (tf.math.argmax(tf.squeeze(phi) ,axis = 0 ), tf.int32 ))
      
      #phi_track.append(phi) 
      #char_track.append(drawing.alphabet[tf.math.argmax(window [0]).numpy().astype(np.int32)] )  
       
          
    
    
      #self.window.assign(window)
      return  (window ,lstm_out ),[ h_lstm, c_lstm , window , kappa_t ]  

 def get_window(self , alpha, beta, kappa ,seq ):
    # phi -> [? x 1 x ascii_steps] and is a tf matrix
    # c -> [? x ascii_steps x alphabet] and is a tf matrix
     ascii_steps = seq.get_shape()[0]  #.value #number of items in sequence
    # print("ascii_steps" , ascii_steps)   
     
     phi = self.get_phi(ascii_steps, alpha, beta, kappa)# phi.shape = (BZ,c.seq,1)
    
    
     window = tf.multiply(phi,seq)
     
     window = tf.reduce_sum(window , 1) # window ~ [?,alphabet]
     #print("window shape after reduce sum: window[0]" , window )
     if periodic_log :  
        print("window shape after reduce sum: window[0]" , window )
        print("c      [0]: " , seq[0] )
        
     return window , phi
 
#get phi for all t,u (returns a [1 x tsteps] matrix) that defines the window
 def get_phi(self ,ascii_steps, alpha, beta, kappa):
        
        
    u = tf.linspace(1.0,tf.dtypes.cast(ascii_steps, tf.float32),ascii_steps )# weight all the U items in the sequence
    u = tf.expand_dims(u , 0)
    u = tf.expand_dims(u , 0)       #(1, 1, ascii_steps)
    u = tf.tile( u , (BATCH_SIZE , self.mixtures , 1))  #(BZ, MIX , ascii_steps)
   
          
    kappa = tf.expand_dims(kappa,2) 
    beta = tf.expand_dims(beta,2)
    alpha = tf.expand_dims(alpha,2)
    phi = alpha * tf.math.exp(tf.math.negative(beta) * tf.math.square(kappa - u))
    
    phi = tf.reduce_sum(phi,1, keepdims=True)

    return tf.reshape( phi ,(BATCH_SIZE ,ascii_steps ,1) )  #  [BS,ascii_steps ,1]
    
 def get_initial_state(self ,inputs, batch_size , dtype):
   
      h_lstm, c_lstm = self.lstm.get_initial_state(inputs, batch_size, dtype)
    
        # to initial kappa value eguals the length of the sequence
      window = tf.zeros((batch_size,vocab_size) , dtype = 'float32')
      init_kappa  = tf.zeros((batch_size,self.mixtures) , dtype = 'float32')  
      return[h_lstm, c_lstm , window , init_kappa]  


In [24]:
class GMM_layer(tf.keras.layers.Layer):
 def __init__(self ,GMM_mixtures):
      super(GMM_layer , self).__init__()
      
#param : activation=tf.nn.relu .will be changed depending on the data set values
#make 'seq_length'param  not hard_coded
      #self.bernoulli0  = tf.keras.layers.Dense(4, activation=tf.nn.sigmoid)
      self.bernoulli  = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,
                                              activation=tf.nn.sigmoid , name = 'bernoulli'))
      
      self.alpha     = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                              GMM_mixtures,name = 'alpha'))
      self.mu_x      = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                                              GMM_mixtures,name = 'mu_s'))#weight.shape(64,20)=>(input,gmm)
      self.mu_y      = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                                              GMM_mixtures,name= 'mu_y'))
      self.sigma_x   = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                                              GMM_mixtures,name ='sigma_x'))
      self.sigma_y   = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                                            GMM_mixtures,name='sigma_y'))
      self.corr      = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(
                                            GMM_mixtures, 
                                             activation=tf.nn.tanh,name = 'corr'))
     
      
 def call(self ,inpt ,mask=None ):
    
    
     self.bernoullis = self.bernoulli(inpt)
     
    
     
     self.alpha_s         =  tf.nn.softmax(self.alpha(inpt) ) 
    
     self.mu_x_s    = self.mu_x(inpt) #(32,1200,20) =>(batch,seq.length,gmm)
     self.mu_y_s    = self.mu_y(inpt)
     #print('self.bernoulli.get_weights()',np.asarray(self.bernoulli.trainable_variables)[0])
   
     self.sigma_xs        =  tf.nn.softplus(self.sigma_x(inpt) - segma_bias)
     self.sigma_ys        =  tf.nn.softplus(self.sigma_y(inpt) - segma_bias)
    
     self.corr_s    = self.corr(inpt)
#      print('self.mu_x_s .shape',self.mu_x_s)
#      print('self.mu_y_s .shape',self.mu_y_s)   
    
     self.bernoulli_distribution = tfd.Bernoulli(probs = self.bernoullis)
     
     self.bi_gaussian = bivariate_gaussian( self.mu_x_s, self.mu_y_s,
                                           self.sigma_xs ,self.sigma_ys , self.corr_s ,self.alpha_s )
     if  periodic_log :
        print('gmm + berll concat',tf.concat([self.bi_gaussian.sample() , 
                       tf.cast(self.bernoulli_distribution.sample(),tf.float32)],axis = -1)[0:1,:,:]
            )
     self.prediction =  tf.concat([self.bi_gaussian.sample() , 
                tf.cast(self.bernoulli_distribution.sample(),tf.float32)],axis = -1)
     
     lst_alpha.append(self.alpha_s)
     lst_logprob .append( self.bi_gaussian.mvd.prob(self.prediction[: ,: ,0:2]))  
     return self.prediction
 
   

In [25]:
class bivariate_gaussian():
    
   def __init__(self , mu1, mu2, sigma1, sigma2, rho, alpha ):
    self. mu1   =  mu1
    self. mu2   =  mu2
    self.sigma1 =  sigma1
    self.sigma2 =  sigma2
    self.rho = rho
    self.alpha = alpha
    
    
    mu  = tf.stack([self.mu1 ,self.mu2] ,axis = -1)   
    
    #print('periodic_log from logprob1',periodic_log)
    if  periodic_log :
       print('mu' , mu)
    sq1 = self.sigma1*self.sigma1
    if  periodic_log :
       print('self.sigma1' , self.sigma1)

    sq2 = self.sigma2*self.sigma2
    if  periodic_log :
        print('self.sigma2' , self.sigma2)

    co = self.rho*self.sigma1*self.sigma2
    if  periodic_log :
        print('co',co)

    row = tf.stack([sq1 , co,co ,sq2] ,axis = -1)
    if  periodic_log :
        print('row     ' ,row)
    self.mvd = tfd.MixtureSameFamily(mixture_distribution=tfd.Categorical(probs=self.alpha),
                                 components_distribution = tfp.distributions.MultivariateNormalTriL(
                                                                   mu,
                       tf.linalg.cholesky(tf.reshape(row ,(BATCH_SIZE ,seq_length ,GMM_mixtures,2,2)))))
    
  
  
   def sample (self) :
    global periodic_log


    smpl =self.mvd.sample()

    if  periodic_log :
        print('smpl' ,smpl[0:5,:,:])
    return smpl 



In [26]:
class rnn_model(tf.keras.Model):
    def __init__(self , enc_units, batch_sz ,kmixtures ,GMM_mixtures  ):
        super(rnn_model , self).__init__()
       
        
       # self.lstm11           = Lstm1(  enc_units, batch_sz)
        self. mask_layer      =  tf.keras.layers.Masking(mask_value=0.,batch_input_shape=(BATCH_SIZE , seq_length, 3))
   
        self.lstm_attention_cell   = lstm_attention_layer(enc_units , kmixtures)
        #self.attention_cell.initial_state()
        self.lstm_att  = tf.keras.layers.RNN(
                                     self.lstm_attention_cell ,stateful = True, return_sequences=True ) 
        #self.rnn              = tf.keras.layers.RNN(self.attention_cell ,  return_sequences=True )
        self.l1 = tf.keras.layers.LSTM(enc_units ,return_sequences = True, 
                                       stateful = True,recurrent_initializer=tf.initializers.GlorotUniform(21),
                                      kernel_initializer=tf.initializers.GlorotUniform(22))
        self.l2 = tf.keras.layers.LSTM(enc_units ,return_sequences = True,
                                       stateful = True,recurrent_initializer=tf.initializers.GlorotUniform(23),
                                      kernel_initializer=tf.initializers.GlorotUniform(24))
  
        #self.lstm22           =   Lstm2(  enc_units )
        
        self.gm_layer         = GMM_layer(GMM_mixtures)
        
    def call (self , inpt ,seq_writer ):
       
#          global periodic_log
#          global first_log
#          global last_batch
        
         
#          if first_log:
#             print("inpt shape" , inpt.shape)  
         
#          start = timer()  
            
#          prev_window = self.attention_cell.return_window() 
         
#          concat_inp = tf.concat([inpt,tf.tile(tf.expand_dims(prev_window,1),(1,1000,1) )] , axis = -1)
#          out1 ,mask   =   self.lstm11(concat_inp)
#          end = timer()
#         # print('lstm1 duration' ,end - start )
        
#          if first_log:
#             print("l1 x shape" , out1.shape)  
            
         start = timer() 
         input_mask =  self. mask_layer( inpt  )
   
         mask = self. mask_layer.compute_mask(inpt)
         end = timer()
        # print('mask  duration' ,end - start )
        
         start = timer() 
         wi , out1   =  self.lstm_att( input_mask ,constants = seq_writer ,mask =mask)
         end = timer()
        # print('attention  duration' ,end - start )
         
            
         if first_log:
            print("window attention shape " , wi.shape)
            print('out1.shape' , out1.shape)
            print("seq shape" , seq_writer[0].shape) 
            print("writer shape" , seq_writer[1].shape) 
         
         #x = tf.concat([x, c], 2) 
         if first_log:
            print("tf.concat([wi,out1,inpt]  x shape" ,tf.concat([wi,out1,inpt] , axis = -1).shape)
            
         start = timer() 
         
         #x   = self.lstm22(tf.concat([wi,out1,inpt] , axis = -1) , mask = mask)
         out2 = self.l1(tf.concat([wi,out1,input_mask] , axis = -1) , mask = mask)
         out3 = self.l2( tf.concat([wi,out2,input_mask] , axis = -1 ), mask = mask)
         end = timer()
         #print('lstm2 duration' ,end - start )
        
         if first_log:
            print("out3  shape" , out3.shape)#(32, 1200, 64)=>(batch,seq.length,unites+)
         
         start = timer()
         self.gm_prediction = self.gm_layer(out3 ,mask = mask)
         end = timer()
         #print('gmm duration' ,end - start )   
            
         if first_log:
            print('gm_prediction' , self.gm_prediction)
         return self.gm_prediction ,mask
  
    def reset(self):
         
         self.lstm_att.reset_states()
         self.l1.reset_states()
         self.l2.reset_states()   

In [27]:
def draw_network_simple_generation0( array):
  for i in range(array.shape[0] ):
    if i%7!=0 :# to get 4 sample
        continue
    fig, ax = plt.subplots(figsize=(12, 3))#np.concatenate([np.cumsum(offsets[:, :2], axis=0), offsets[:, 2:3]], axis=1)
    #plt.ylim((-50,50))
    plt.title('network_out')
    
    print('array ',array.shape)
   
    one_net_strokes = array[i ,:,:]
    
    one_net_strokes =  tf.split(tf.transpose(one_net_strokes),3 ,axis =0)
    one_net_strokes = np.asarray(one_net_strokes)
    one_net_strokes =np.concatenate( [np.cumsum(one_net_strokes[:2, : , :], axis=2),
                                       one_net_strokes[2:,: ,:]], axis=0)
   
    end_of_stroke =0
    
    stroke = [(0,0)]
   
    one_net_strokes = tf.squeeze(one_net_strokes)
    one_net_strokes = tf.transpose(one_net_strokes)
       
    one_net_strokes= tf.dtypes.cast(one_net_strokes, tf.float32)
    for indx , triple_coor in enumerate ( one_net_strokes):
       
        stroke.append((triple_coor[0], triple_coor[1]))
        if ((triple_coor[2].numpy() == 1.0) or indx + 1 == len(one_net_strokes)) :
            
            coords = list(zip(*stroke))
            end_of_stroke +=1
            ax.plot(coords[0], coords[1], '-k')
            
            stroke = []
    print('end of stroke',end_of_stroke)
    
#draw_network_simple_generation( rnn1.gm_prediction)    

In [28]:
rnn11 = rnn_model( units , BATCH_SIZE , k_mixture ,GMM_mixtures  )

In [29]:
rnn11.load_weights(path_weight)

In [30]:
def condition_to_terminate(coor ,phi ,len_of_char) :
    
    current_attention = phi
    if_afler_last_char = current_attention >= len_of_char - 1
    
    #print('current_attention {} ;len_of_char {}'.format(current_attention , len_of_char) )
    
    return ( if_afler_last_char  and  (coor[ 0 , 0 , 2] == 1) ) or (current_attention > len_of_char - 1) # 

In [31]:
def first_input(batch_size) :
    i = [0. ,0. , 1.] * batch_size
    i = tf.constant(i)
    i = tf.reshape(i , (batch_size ,seq_length , 3))
    
    return i
first_in =   first_input(BATCH_SIZE)
first_in

<tf.Tensor: shape=(1, 1, 3), dtype=float32, numpy=array([[[0., 0., 1.]]], dtype=float32)>

In [32]:
np.random.randint(0,number_of_writer , 1)

array([66])

In [33]:
writer_matrix = tf.constant( np.identity(number_of_writer , dtype=np.float32)) 

def random_writer():
   
    random_w = np.random.randint(0,number_of_writer , 1)
    list_writer = writer_matrix[random_w[0]]
    return tf.reshape(list_writer , (1,number_of_writer))


In [34]:
  #list of synthesis params :( segma_bias , alpha_bias , number of samples)
  bias_numbers =  [(1., 5),(5., 5)]
  
  segma_bias =  tf.Variable(0 , dtype='float32' , trainable = False) 
  
  alpha_bias =  tf.Variable(0 , dtype='float32' , trainable = False) 
  
 #variable to track the attention( on char sequence of string to write) to stop synthesis of points;used in function "condition_to_terminate"
  new_phi = tf.Variable( 0,  dtype='int32' ,trainable = False)

In [35]:
# generate a number "num_of_sample" of words "real_seq" with the same params(biases)
def hand_synthesis(real_seq ,num_of_sample ):
    
    global phi_track 
    global char_track
    global rnn11
    
    list_of_handwrite = []
   
    
    
    for i in tf.range(num_of_sample) :
      seq_to_write = real_seq #tf.expand_dims( , 0)
     
      writer =  random_writer()
      len_of_char  = seq_to_write.shape[0]
      
      inp = first_in
      phi_track = []
      char_track = []
      coordList = []
        

      while True:  #if  condition_to_terminate is false , generate an other coordinate  
           
            coord , _ = rnn11(inp  ,[seq_to_write , writer])  #synt_step
            inp    = coord                                           #(1,1,3)
            coordList.append(coord )
           
            if  condition_to_terminate(coord , new_phi ,len_of_char )  or len(coordList) > 750 :
                break
      rnn11.reset( )
      
      out = tf.reshape(tf.stack( coordList), [1, len(coordList), 3])   
      list_of_handwrite.append(out)   
     

        
    
    return list_of_handwrite

    

In [36]:


def generated_to_png( list_of_handwrite  , seq_write ):
 
 
 for   array in list_of_handwrite :
    
    stroke = []
    
    stringToWrite =  tostring (seq_write)
    
    fig, ax = plt.subplots(figsize=(10, 3))   
    
    one_net_strokes = tf.squeeze(array )
    #print('one_net_strokes' , one_net_strokes)
    
    one_net_strokes =tf.concat( [tf.cumsum(one_net_strokes[:, :2 ], axis=0),
                                       one_net_strokes[:, 2: ]], axis=1)
   
    
    #  Matrix transormation to add skew to letters( by degree ) : 
    
#     theta = degree * np.pi/180
#     A = np.array([[np.cos(-theta), 0], [np.sin(-theta), 1]])
#     one_net_strokes[:, :2] = np.dot(one_net_strokes[:, :2], A) 
    
   
    for indx , triple_coor in enumerate ( one_net_strokes):
    
        
        stroke.append((triple_coor[0], triple_coor[1]))
        
        if ((triple_coor[2] == 1.0  and indx != 0 ) or indx + 1 == len(one_net_strokes)) :
            
            coords = list(zip(*stroke))

            plt.axis('off')
            
            ax.plot(coords[0], coords[1], '-k')
            
            stroke = []
        
    save_png( plt , stringToWrite )
    plt.close('all')


In [37]:

def save_png(plt , stringToWrite ) :
    pt = path_tosave_image + stringToWrite.strip()
   
    pathlib.Path(pt).mkdir(parents=True, exist_ok=True) 
    
    cpt = sum([len(files) for _ ,_ , files in os.walk(pt)])
    
    plt.savefig( pt+ '/' + str(cpt ) +'.png' ,  bbox_inches = 0 )
    

In [38]:

def tostring (seq_write) :
    
    rrrr=tf.math.argmax(seq_write , axis = -1)#.numpy().astype(np.int32)
    
    ooo = [[drawing.alphabet[n] for n in i if n !=0]for i in rrrr]
    
    stringToWrite = [ ''.join(k) for k in ooo]
    return stringToWrite[0] 
 

In [39]:
  start0 = timer()
    
#the handwriting syntesis was stopped at index 50 .lastIndex = 50 
  param_counter = 0
  for s_bias  , num in bias_numbers :           # loop on each param of handwrting syntesis : bias
    
    segma_bias.assign(s_bias)
   
    number_of_sample = num
    
    first_log = False 
    for indx ,  med in enumerate(real_seqs):     #get handwriting for each med_name in list of tensors
       
    
      if indx < lastIndex and s_bias == 1.0 :
            continue
            
       start = timer()
       list_of_handwrite = hand_synthesis(med, number_of_sample )
   
       generated_to_png( list_of_handwrite  ,tf.expand_dims(med , 0))
       if indx % 10 == 0 :
            tf.print(indx)
            end = timer()
            tf.print('syntesis duration of string is' ,end - start )
    param_counter += 1   
       
  end0 = timer()
  print('syntesis duration' ,end0 - start0 )    

0
syntesis duration of string is 41.44259749999992
10
syntesis duration of string is 68.36862829999995
20
syntesis duration of string is 29.975505599999906
30
syntesis duration of string is 63.19640530000015
40
syntesis duration of string is 34.07274009999992
50
syntesis duration of string is 35.521002299999964


KeyboardInterrupt: 

In [47]:
# real_seqs_np = [t.numpy() for t in real_seqs]
# np.save('real_seqs_medicament2_saved.npy' ,real_seqs_np ,allow_pickle=True)

In [ ]:
len(phi_track)

In [ ]:
phi_plt = tf.convert_to_tensor(phi_track)
phi_plt = tf.squeeze(phi_plt)
phi_plt = tf.transpose(phi_plt)

In [ ]:
#plt.plot(tf.math.argmax(tf.squeeze(tf.transpose(phi_track)) ,axis = 0 ))
#plt.axis([0, 1, 0, 10000])
fig, ax = plt.subplots()

list_chara = [v for i, v in enumerate(char_track) if i == 0 or v != char_track[i-1]]
ax.set_yticks(np.arange(0,len(list_chara)))

ax.set_yticklabels(list_chara)
im = ax.imshow(phi_plt , aspect='auto')


In [ ]:
len(char_track)

In [ ]:
plt.plot(char_track)
#plt.axis([0, 1, 0, 10000])
plt.show()

In [ ]:
toTensor = tf.convert_to_tensor(lst_alpha)
toTensor =tf.squeeze( toTensor)


In [ ]:
toTensor.shape

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches (30,200)
# plt.xlabel('time step')
# plt.ylabel('20 gmm activation')
im = plt.imshow(tf.transpose(toTensor))


In [ ]:
handw = tf.convert_to_tensor(list_of_handwrite)
hand = tf.squeeze(handw)
hand = hand[: , 0:2]
handx  = hand[: , 0]*100
handy = hand[: , 1]*100

handx = tf.cumsum(handx , axis=0)
handy = tf.cumsum(handy , axis=0)
# handx  = tf.clip_by_value(handx ,clip_value_min = -200 , clip_value_max = 200)
# handy  = tf.clip_by_value(handy , clip_value_min = -250 ,clip_value_max = 250)
handx.shape

In [ ]:
lst_logprob1 = tf.convert_to_tensor(tf.math.log(lst_logprob))
lst_logprob1 = tf.squeeze(lst_logprob1)
lst_logprob1

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches (80,20)
ax.set_facecolor("black")
plt.scatter(handx, handy, s= 128000* (1/lst_logprob1.numpy() )**3  ,  facecolor='yellow' )


In [ ]:
rnn11.summary()